##  요청 사항
* IndySDK 설치 요청

## 추가구현
### 동작
#### TBD
* 물체 방향 따라 근접샷 자세 잡기
* 중간 감지 결과 따라 로봇을 움직이기 - 병합/실험 시 적용
* 툴 각도 맞추기 - 요청
* 추가요소
  * ros로 맵 정보 얻기 - 목표점 충돌 가능성 확인 필요
    - master를 ip 통해서 지정 - 일단 방법 숙지
  * 옆으로 동작하기

#### DONE
* reach data 수집 시 모바일 베이스 위에서 수집, 모바일 앞쪽 충돌 경계랑 베드 높이랑 맞춰서 수집해놓기 v
* 닦기 순서 바꾸기 - 옷장 -> 베드 v
* 스케줄간 동작 연결 v
  
### 인식
* 정면 인식 불확실성 해결
  - 제일 가까운 포인트를 침대 다리부분으로 가정, 바닥 각도만 맞춰서 초기화 하고 ICP 시도 + 기본 ICP 해서 더 잘 맞은 쪽 선택하기 시도
* 옷장 근접 인식 - 부분 피팅 + 방향 에러 -> 위치 에러
  - 찍힌 부분 중심 Transformation
  - 스캔 다 해서 피팅
* 침대 인식 부정확 - 전반적으로 홈 위치에 가깝게 인식되는 경향 있음
* 추가 요소
  * 전체 환경 reconsruction에 촬영 이미지를 추가 + 현재 위치 맞추는 방식
    - 한 요소 이동 방식으로는 두 요소간 위치 관계 반영 안됨 (한개 중심으로 맞추면 다른게 틀어짐)